In [1]:
!pip install transformers==4.27.2
!pip install datasets
!pip install evaluate
!pip install rouge_score
%pip install loralib \
                peft --quieta
!pip install accelerate -U
!pip install --upgrade safetensors


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=4f22e87ec19a959cfc7a1b2ce7398798abb0f81a055113d5cbbf3de2fc432ee4
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score

Usage:   
  pip3 

In [2]:
import torch
import torchdata
from transformers import AutoModelForSeq2SeqLM , AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
from  datasets import load_dataset
import evaluate
import time
import pandas as pd
import numpy as np

In [3]:
import transformers
transformers.__version__

'4.27.2'

In [4]:
#Load DialogSum Hugging face Dataset
hugginface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(hugginface_dataset_name)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:


#load Model and Tokenizer

model_name = 'google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype = torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [6]:
#pull out total number of weights parameter and find out how many are trainable

def print_number_of_trainable_parameter(model):
  trainable_model_param = 0
  all_model_parameter = 0
  for _, param in model.named_parameters():
    all_model_parameter += param.numel()
    if param.requires_grad:
      trainable_model_param += param.numel()
  return f"Trainable model paramtere: {trainable_model_param} \n all model parameter: {all_model_parameter}\n precentage of trainable model parameter :{100*(trainable_model_param/all_model_parameter)}%"

In [7]:
print(print_number_of_trainable_parameter(original_model))

Trainable model paramtere: 247577856 
 all model parameter: 247577856
 precentage of trainable model parameter :100.0%


In [8]:
#test the model with zero shot inferencing
index = 200

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.
{dialogue}
Summary:
"""

input = tokenizer(prompt, return_tensors = 'pt')
output = tokenizer.decode(
    original_model.generate(input["input_ids"],
                            max_new_tokens = 200,
                            )[0], skip_special_tokens = True
)

dash_line = '_'.join('' for x in range(100))
print(dash_line)
print(f"input prompt : {prompt}")
print(dash_line)
print(f"BASELINE HUMAN SUMMARY : {summary}")
print(dash_line)
print(f"MODEL_GENERETED OUTPUT: {output}")

___________________________________________________________________________________________________
input prompt : 
Summarize the following conversation.
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.
Summary:

____________________________________________________________________

# **Perform Full Fine-Tuning**
#### **preprocess the Dialogue Summary Dataset**

You need to convert dialogue summary pair into explicit instruction for the LLM. Prepend Instruction to start the dialogue with "Summarize the following conversation" and to the start of the summary with Summary as follows:

In [9]:
def tokenize_funtion(example):
  start_prompt = 'Summarize the following conversation.\n\n'
  end_prompt = "\n\n Summary "

  prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
  example["input_ids"] = tokenizer(prompt, padding = "max_length",truncation = True, return_tensors = "pt").input_ids
  example["labels"] = tokenizer(example['summary'], padding = "max_length", truncation = True, return_tensors = "pt").input_ids

  return example

In [10]:
tokenized_dataset = dataset.map(tokenize_funtion, batched = True)
tokenized_dataset = tokenized_dataset.remove_columns(['id', 'topic','dialogue','summary',])

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [11]:
#subsample the dataset
tokenized_dataset = tokenized_dataset.filter(lambda example, index: index % 10000 == 0, with_indices = True)

Filter:   0%|          | 0/12460 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [12]:
print("Shape of the datasets")
print(f"training: {tokenized_dataset['train'].shape}")
print(f"Validation: {tokenized_dataset['validation'].shape}")
print(f"Test: {tokenized_dataset['test'].shape}")

Shape of the datasets
training: (2, 2)
Validation: (1, 2)
Test: (1, 2)


# Fine Tune the Model with Preprocessed Dataset using PEFT

In [36]:
import accelerate
accelerate.__version__

'0.23.0'

In [37]:
 !pip install git+https://github.com/huggingface/accelerate

  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-wy8ftv7h
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-wy8ftv7h
  Resolved https://github.com/huggingface/accelerate to commit a87c95da9e3b416fb10a0e7dac7d397c015c3ed5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.24.0.dev0-py3-none-any.whl size=258134 sha256=8ebf9cdd124b7b923d50f9f3acf95272ed304e55d0a36887c2c157128f226549
  Stored in directory: /tmp/pip-ephem-wheel-cache-4ywr3nlx/wheels/f6/c7/9d/1b8a5ca8353d9307733bc719107acb67acdc95063bba749f26
Successfully built accelerate
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.23.0
    Uninstalling accelerate-0.23.0:
      Successfully uninstalled accelerate-0.23.0


In [38]:
output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'

train_arg = TrainingArguments(
    output_dir = output_dir,
    learning_rate = 1e-5,
    num_train_epochs = 1,
    weight_decay = 0.01,
    logging_steps = 1,
    max_steps=1
)

trainer = Trainer(
    model = original_model,
    args = train_arg,
    train_dataset = tokenized_dataset['validation'],
    eval_dataset = tokenized_dataset['validation']
)

In [39]:
#train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1,48.750000


TrainOutput(global_step=1, training_loss=48.75, metrics={'train_runtime': 461.298, 'train_samples_per_second': 0.017, 'train_steps_per_second': 0.002, 'total_flos': 695628988416.0, 'train_loss': 48.75, 'epoch': 1.0})

In [41]:
instruct_model_path="./instruct-dialogue-summary-checkpoint-local"
trainer.model.save_pretrained(instruct_model_path)
tokenizer.save_pretrained(instruct_model_path)

('./instruct-dialogue-summary-checkpoint-local/tokenizer_config.json',
 './instruct-dialogue-summary-checkpoint-local/special_tokens_map.json',
 './instruct-dialogue-summary-checkpoint-local/tokenizer.json')

In [42]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained(instruct_model_path, torch_dtype=torch.bfloat16)

Some weights of the model checkpoint at ./instruct-dialogue-summary-checkpoint-local were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.v.lora_A.default.weight', 'decoder.block.11.layer.1.EncDecAttention.v.lora_A.default.weight', 'decoder.block.2.layer.0.SelfAttention.v.lora_A.default.weight', 'encoder.block.9.layer.0.SelfAttention.v.lora_B.default.weight', 'decoder.block.7.layer.0.SelfAttention.v.lora_B.default.weight', 'decoder.block.3.layer.0.SelfAttention.q.lora_B.default.weight', 'decoder.block.7.layer.1.EncDecAttention.q.lora_B.default.weight', 'decoder.block.0.layer.1.EncDecAttention.v.lora_B.default.weight', 'encoder.block.4.layer.0.SelfAttention.q.lora_A.default.weight', 'decoder.block.8.layer.0.SelfAttention.q.lora_A.default.weight', 'decoder.block.4.layer.0.SelfAttention.q.lora_A.default.weight', 'decoder.block.2.layer.1.EncDecAttention.q.lora_B.default.weight', 'decoder.block.11.layer.0.SelfAttention.v.lora_A.default.weight

In [43]:
index = 200
dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following converation.
{dialogue}
Summary
"""

inputs = tokenizer(prompt, return_tensors = 'pt').input_ids

instruct_model_outputs = instruct_model.generate(input_ids = inputs, generation_config = GenerationConfig(max_new_tokens = 200, temprature= 0))

instruct_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens= True)

original_model_output = tokenizer.decode(
    original_model.generate(input_ids = inputs,
                            generation_config = GenerationConfig(max_new_tokens = 200, temprature= 0)
                            )[0],

    skip_special_tokens = True
)

print(instruct_model_text_output)

print(dash_line)

print(summary)

print(dash_line)

print(original_model_output)


#Person1#: You might want to upgrade your system. #Person2#: You might want to upgrade your hardware. #Person1#: You might want to upgrade your computer.
___________________________________________________________________________________________________
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
___________________________________________________________________________________________________
#Person1#: You could add a painting program to your software.


In [19]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 2.1 MB/s eta 0:00:00


In [20]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r = 32, #rank
    lora_alpha = 32,
    target_modules = ["q","v"],
    lora_dropout = 0.05,
    bias = "none",
    task_type = TaskType.SEQ_2_SEQ_LM #Flan t-5
)

In [21]:
# add Lora Adapter Layers into original LLM Model to be trained
peft_model = get_peft_model(original_model, lora_config)
print(print_number_of_trainable_parameter(peft_model))

Trainable model paramtere: 3538944 
 all model parameter: 251116800
 precentage of trainable model parameter :1.4092820552029972%


In [22]:
#Train peft adapter
output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'
peft_training_arguments = TrainingArguments(
    output_dir = output_dir,
    auto_find_batch_size = True,
    learning_rate =  1e-3,
    num_train_epochs = 1,
    logging_steps = 1,
    max_steps = 1
)

peft_trainer = Trainer(
    model = peft_model,
    args = peft_training_arguments,
    train_dataset = tokenized_dataset["train"]

)

In [23]:
peft_trainer.train()
peft_model_path="./peft-dialogue-summary-checkpoint-local"
peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1,46.500000


('./peft-dialogue-summary-checkpoint-local/tokenizer_config.json',
 './peft-dialogue-summary-checkpoint-local/special_tokens_map.json',
 './peft-dialogue-summary-checkpoint-local/tokenizer.json')

In [24]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype = torch.bfloat16)

tokenizer = AutoTokenizer.from_pretrained(model_name)

peft_model = PeftModel.from_pretrained(peft_model_base,peft_model_path,torch_type = torch.bfloat16,is_trainable = False)

In [25]:
print(print_number_of_trainable_parameter(peft_model))

Trainable model paramtere: 0 
 all model parameter: 251116800
 precentage of trainable model parameter :0.0%


In [26]:
index = 200
dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following converation.
{dialogue}
Summary
"""

inputs = tokenizer(prompt, return_tensors = 'pt').input_ids

peft_model_outputs = peft_model.generate(input_ids = inputs, generation_config = GenerationConfig(max_new_tokens = 200, temprature= 0))

peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens= True)

original_model_output = tokenizer.decode(
    original_model.generate(input_ids = inputs,
                            generation_config = GenerationConfig(max_new_tokens = 200, temprature= 0)
                            )[0],

    skip_special_tokens = True
)

print(peft_model_text_output)

print(dash_line)

print(summary)

print(dash_line)

print(original_model_output)


#Person1#: You might want to upgrade your system. #Person2#: You might want to upgrade your hardware. #Person1#: You might want to upgrade your computer.
___________________________________________________________________________________________________
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
___________________________________________________________________________________________________
#Porderedo - I'm not sure what exactly I would need to upgrade my computer. #Person1#: I'm not sure what I would need to upgrade my hardware. #Person1#: I'm not sure what I would need to upgrade my computer. #Person1#: I'm not sure what I would need to upgrade my computer.


#Evaluate the Model Qunatitevely with Metric (Rouge Method)

In [48]:
dailogues = dataset['test'][0:10]['dialogue']
human_base_summary = dataset['test'][0:10]['summary']

original_model_summary = []
peft_model_summary = []
instruct_model_summary = []
for idx, dialogue in enumerate(dailogues):
  prompt = f"""
           Summarize the following conversation:
           {dialogue}
           Summary:
  """

  inputs = tokenizer(prompt , return_tensors = 'pt').input_ids
  #original mode
  original_model_output = original_model.generate(input_ids = inputs , generation_config = GenerationConfig(max_new_tokens = 50))
  original_model_output_text = tokenizer.decode(original_model_output[0] , skip_speacial_tokens = True)

  #Instruct Model
  instruct_model_output =  instruct_model.generate(input_ids = inputs, generation_config = GenerationConfig(max_nw_toke = 50))
  instruct_model_output_text = tokenizer.decode(instruct_model_output[0],skip_speacial_tokens = True)
  #peft Model
  peft_model_output = peft_model.generate(input_ids = inputs , generation_config = GenerationConfig(max_new_tokens = 50))
  peft_model_output_text = tokenizer.decode(peft_model_output[0] , skip_special_tokens = True)

  original_model_summary.append(original_model_output_text)
  peft_model_summary.append(peft_model_output_text)
  instruct_model_summary.append(instruct_model_output_text)

zipped_summarize = list(zip(human_base_summary,original_model_summary, peft_model_summary,instruct_model_summary))
df = pd.DataFrame(zipped_summarize, columns = ['Human Base Summary', 'Original Model Summary','instruct_model_output_text', 'Peft Model Summary'])
df

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


,Human Base Summary,Original Model Summary,instruct_model_output_text,Peft Model Summary
0,Ms. Dawson helps #Person1# to write a memo to ...,<pad> The memo is going out as an intra-office...,#Person1#: I need to take a dictation for you.,<pad> #Person1#: I need to take a dictation fo...
1,In order to prevent employees from wasting tim...,<pad> The memo should go out by this afternoon...,#Person1#: I need to take a dictation for you.,<pad> #Person1#: I need to take a dictation fo...
2,Ms. Dawson takes a dictation for #Person1# abo...,<pad> This memo is to be distributed to all em...,#Person1#: I need to take a dictation for you.,<pad> #Person1#: I need to take a dictation fo...
3,#Person2# arrives late because of traffic jam....,<pad> Taking public transport to work is a goo...,The traffic jam at the Carrefour intersection ...,<pad> The traffic jam at the Carrefour interse...
4,#Person2# decides to follow #Person1#'s sugges...,<pad> I'm a car driver.</s>,The traffic jam at the Carrefour intersection ...,<pad> The traffic jam at the Carrefour interse...
5,#Person2# complains to #Person1# about the tra...,<pad> People are talking about the traffic jam...,The traffic jam at the Carrefour intersection ...,<pad> The traffic jam at the Carrefour interse...
6,#Person1# tells Kate that Masha and Hero get d...,<pad> Masha and Hero are getting divorced.</s>,Masha and Hero are getting divorced.,<pad> Masha and Hero are getting divorced.</s>
7,#Person1# tells Kate that Masha and Hero are g...,"<pad> #Person1#: You are always right, Masha a...",Masha and Hero are getting divorced.,<pad> Masha and Hero are getting divorced.</s>
8,#Person1# and Kate talk about the divorce betw...,<pad> The divorce papers are filed for the kid...,Masha and Hero are getting divorced.,<pad> Masha and Hero are getting divorced.</s>
9,#Person1# and Brian are at the birthday party ...,"<pad> #Person1: Happy Birthday, Brian. #Person...","#Person1#: Happy Birthday, Brian. #Person2#: I...","<pad> #Person1#: Happy Birthday, Brian. #Perso..."


In [51]:
rouge = evaluate.load('rouge')

original_model_result = rouge.compute(
    predictions = original_model_summary,
    references = human_base_summary[0:len(original_model_summary)],
    use_aggregator = True,
    use_stemmer = True

)

intruct_model_result = rouge.compute(
    predictions = instruct_model_summary,
    references = human_base_summary[0:len(original_model_summary)],
    use_aggregator = True,
    use_stemmer = True
)

peft_model_result = rouge.compute(
    predictions = peft_model_summary,
    references = human_base_summary[0:len(original_model_summary)],
    use_aggregator = True,
    use_stemmer = True
)

In [52]:
print(f'ORIGINAL MODEL : {original_model_result}')
print(f'Peft MODEL RESULT : {peft_model_result}')
print(f'INSRUCT MODEL RESULT : {intruct_model_result}')

ORIGINAL MODEL : {'rouge1': 0.21691443027791496, 'rouge2': 0.08097463768115941, 'rougeL': 0.1893890451244437, 'rougeLsum': 0.19190725216365648}
Peft MODEL RESULT : {'rouge1': 0.29494370578091506, 'rouge2': 0.11442557875058065, 'rougeL': 0.253800262056076, 'rougeLsum': 0.25717517543098933}
INSRUCT MODEL RESULT : {'rouge1': 0.2851961047546647, 'rouge2': 0.1045014245014245, 'rougeL': 0.2445031830738451, 'rougeLsum': 0.24720154316253998}


In [53]:
print("Absolute percentage improvement of PEFT MODEL over HUMAN BASELINE")

improvement = (np.array(list(peft_model_result.values())) - np.array(list(original_model_result.values())))
for key, value in zip(peft_model_result.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of PEFT MODEL over HUMAN BASELINE
rouge1: 7.80%
rouge2: 3.35%
rougeL: 6.44%
rougeLsum: 6.53%


In [54]:
print("Absolute percentage improvement of INSRUCT MODEL over HUMAN BASELINE")

improvement = (np.array(list(intruct_model_result.values())) - np.array(list(original_model_result.values())))
for key, value in zip(intruct_model_result.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of INSRUCT MODEL over HUMAN BASELINE
rouge1: 6.83%
rouge2: 2.35%
rougeL: 5.51%
rougeLsum: 5.53%
